In [135]:
import pandas as pd
import numpy as np
import random 
np.random.seed = 42
random.seed(42)

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 100)
num_ofDesiredRecords = 4000

In [136]:
def createRandomPhoneNumber_SAA():
    range_start = 10**9
    range_end = 10**10 -1
    num = str(random.randint(range_start,range_end))
    num_str = "{0} {1}-{2}".format(num[:3], num[3:7],num[7:])
    return num_str

In [137]:
def createRandomEmail(record,domainList=['@gmail.com','@stanfordalumni.org','@alumni.stanford.edu']):
    domain = random.choice(domainList)
    fName = record['first_name'].strip().lower()
    LName = record['last_name'].strip().lower()
    formats = []
    
    #Example: John Doe 123
    formats.append(fName[0]+LName+str(random.randint(10,9999))) #jdoe123
    formats.append(fName+LName+str(random.randint(10,9999))) #johndoe123
    formats.append(fName+LName[0]+str(random.randint(10,9999))) #johnd123
    formats.append(fName[0]+'.'+LName+str(random.randint(10,9999))) #j.doe123
    formats.append(fName+'.'+LName+str(random.randint(10,9999))) #john.doe123
    
    return random.choice(formats)+domain

In [138]:
def createDegreeString_SAA():
    degree_list = ['MS','MA','MBA','MD','PhD','BA','BS','JD','']
    degree_years = "'" + str(random.randint(80,99))
    degree = random.choice(degree_list) + ' ' + degree_years
    return degree.strip()

In [139]:
common_male_Fnames = [
    'Liam',
    'Noah',
    'Oliver',
    'Elijah',
    'James',
    'William',
    'Benjamin',
    'Lucas',
    'Henry',
    'Theodore',
]

In [140]:
common_female_Fnames = [
    'Mary',
    'Patricia',
    'Jennifer',
    'Linda',
    'Elizabeth',
    'Barbara',
    'Susan',
    'Jessica',
    'Sarah',
    'Karen',
]

In [141]:
city = ['Chicago', 'Boston',  'Madrid', 'Tokyo', 'Seoul', 'London','Beijing','Shanghai','Dubai','*',np.nan,'']
state = ['NY', 'WA', 'TX','CA','NM','*',np.nan,'']
country = ['Japan', 'United States', 'USA', 'China', 'UAE', 'Kuwait','*',np.nan,'']

In [142]:
common_FNames = []
common_FNames.extend(common_female_Fnames)
common_FNames.extend(common_male_Fnames)

In [143]:
common_LNames = ['Smith',
'Johnson',
'Williams',
'Brown',
'Jones',
'Garcia',
'Miller',
'Davis',
'Rodriguez',
'Martinez',]

In [144]:
df_SAA = pd.read_excel("./data/SAA Pride member reports headings.xlsx").append([np.nan]*num_ofDesiredRecords)

C:\Users\saads\AppData\Local\Temp\ipykernel_10524\2961325751.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SAA = pd.read_excel("./data/SAA Pride member reports headings.xlsx").append([np.nan]*num_ofDesiredRecords)


In [145]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(common_FNames))
    
df_SAA.first_name = __

In [146]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(common_LNames))
    
df_SAA.last_name = __

In [147]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choices([createRandomPhoneNumber_SAA(),'*',np.nan],weights=(1,5,5))[0])
    
df_SAA.home_phone_number = __

In [148]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(['*',np.nan,createRandomEmail(df_SAA.iloc[_])]))
    
df_SAA.home_email_address = __

In [149]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(['*',np.nan,createRandomEmail(df_SAA.iloc[_],['@stanfordalumni.org','@alumni.stanford.edu'])]))
    
df_SAA.saa_email_address = __

In [150]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choices([createRandomEmail(df_SAA.iloc[_]),np.nan],weights=(1, 50))[0])
    
df_SAA.gsb_email_address = __

In [151]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choices([createRandomEmail(df_SAA.iloc[_]),np.nan],weights=(1, 10))[0])
    
df_SAA.bus_email_address = __

In [152]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(city))
    
df_SAA.home_city = __

In [153]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(country))
    
df_SAA.home_country = __

In [154]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(country))
    
df_SAA.bus_country = __

In [155]:
state = {'Chicago': 'IL', 
           'Boston': 'MA', 
           'New York' : 'NY', 
           'San Francisco': 'CA', 
           'Los Angeles' : 'CA', 
           'Austin' : 'TX',
        'Dallas': 'TX',
        'Denver': 'CO',
        '':'',
        '*':'*'}

df_SAA.home_state_code = df_SAA['home_city'].map(state)

In [156]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(['',np.nan,random.randint(1990, 2018)]))
    
df_SAA.pref_class_year = __


In [157]:
degrees = []
for i in range(0,num_ofDesiredRecords):
    num_draw = random.randint(0,3)
    degree = np.NaN
    k=0
    while k < num_draw:
        if k == 0:
            degree = createDegreeString_SAA()
        else:
            degree = degree +', '+ createDegreeString_SAA()
        k+=1
    degrees.append(degree)
    
df_SAA.short_degree_string = degrees

In [27]:
# df_SAA.to_excel('data/SAA_FakeDB.xlsx',index = False)

# Making mail chimp records

In [179]:
df_mc = pd.read_csv('data/MailChimp cleaned records headers.csv')
df_mc

,Email Address,First Name,Last Name,Board Member,Gender,Chapter,Reunion Year,Country,Degree,MEMBER_RATING,OPTIN_TIME,OPTIN_IP,CONFIRM_TIME,CONFIRM_IP,LATITUDE,LONGITUDE,GMTOFF,DSTOFF,TIMEZONE,CC,REGION,CLEAN_TIME,CLEAN_CAMPAIGN_TITLE,CLEAN_CAMPAIGN_ID,LEID,EUID,NOTES,TAGS


In [180]:
df_mc = df_mc.append([np.nan]*5)

C:\Users\saads\AppData\Local\Temp\ipykernel_10524\705722782.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_mc = df_mc.append([np.nan]*5)


In [181]:
df_SAA[df_SAA.first_name=='Barbara']

,pref_mail_name,pref_class_year,home_city,home_state_code,home_country,home_phone_area_code,home_phone_number,home_email_address,bus_city,bus_state_code,bus_country,bus_phone_area_code,bus_phone_number,bus_email_address,first_name,last_name,pref_name_sort,email_switch,saa_email_address,gsb_email_address,other_email_address,pref_phone_area_code,pref_phone_number,pref_phone_addr_type,memb_status_desc,short_degree_string,parent_degree_string,short_degree_string_spouse,parent_degree_string_spouse,primary_sort_name,plan_name,primary_ind,0
28,NaN,2003.0,Dubai,NaN,*,NaN,NaN,barbara.smith9669@gmail.com,NaN,NaN,Japan,NaN,NaN,NaN,Barbara,Smith,NaN,NaN,b.smith5277@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,Boston,MA,NaN,NaN,NaN,NaN,NaN,NaN,Kuwait,NaN,NaN,bjohnson6108@stanfordalumni.org,Barbara,Johnson,NaN,NaN,*,barbaraj9242@gmail.com,NaN,NaN,NaN,NaN,NaN,'93,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NaN,1991.0,Dubai,NaN,USA,NaN,*,brodriguez369@stanfordalumni.org,NaN,NaN,United States,NaN,NaN,NaN,Barbara,Rodriguez,NaN,NaN,barbara.rodriguez5922@stanfordalumni.org,brodriguez2537@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,NaN,NaN,NaN,NaN,UAE,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,Barbara,Rodriguez,NaN,NaN,barbara.rodriguez2112@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,'97,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
# First record. No major difference other than email
rec = df_SAA.loc[73]
i = 0
df_mc.loc[i,'First Name'] = rec.first_name
df_mc.loc[i,'Last Name'] = rec.last_name
df_mc.loc[i,'Email Address'] = rec.home_email_address.split('@')[0]+'@gmail.com'
df_mc.loc[i,'Country'] = rec.home_country
df_mc.loc[i,'Degree'] = random.choice([np.nan,''])

df_mc.loc[i,'Board Member'] = random.choice([True,False])
df_mc.loc[i,'Gender'] = random.choice(['F','M',np.nan])
df_mc.loc[i,'Chapter'] = random.choice(['Other US','Texas','Bay Area','DC Area','New England'])

In [183]:
# Second record. Missing all degrees. Country USA. Different email
rec = df_SAA.loc[34]
i=1

df_mc.loc[i,'First Name'] = rec.first_name
df_mc.loc[i,'Last Name'] = rec.last_name
df_mc.loc[i,'Email Address'] = createRandomEmail(rec)
df_mc.loc[i,'Country'] = 'USA'
df_mc.loc[i,'Degree'] = random.choice([np.nan,''])

df_mc.loc[i,'Board Member'] = random.choice([True,False])
df_mc.loc[i,'Gender'] = random.choice(['F','M',np.nan])
df_mc.loc[i,'Chapter'] = random.choice(['Other US','Texas','Bay Area','DC Area','New England'])

# Third record. Missing 1 degree. Country USA. Different email
rec = df_SAA.loc[30]
i=2
df_mc.loc[i,'First Name'] = rec.first_name
df_mc.loc[i,'Last Name'] = rec.last_name
df_mc.loc[i,'Email Address'] = 'lindam7966@alumni.stanford.edu'
df_mc.loc[i,'Country'] = 'USA'
df_mc.loc[i,'Degree'] = 'MD;MA'

df_mc.loc[i,'Board Member'] = random.choice([True,False])
df_mc.loc[i,'Gender'] = random.choice(['F','M',np.nan])
df_mc.loc[i,'Chapter'] = random.choice(['Other US','Texas','Bay Area','DC Area','New England'])

In [184]:
# 4th record. Has a degree on mail chimp side
rec = df_SAA.loc[92]
i=3
df_mc.loc[i,'First Name'] = rec.first_name
df_mc.loc[i,'Last Name'] = rec.last_name
df_mc.loc[i,'Email Address'] = createRandomEmail(rec)
df_mc.loc[i,'Country'] = 'USA'
df_mc.loc[i,'Degree'] = 'MS'

df_mc.loc[i,'Board Member'] = random.choice([True,False])
df_mc.loc[i,'Gender'] = random.choice(['F','M',np.nan])
df_mc.loc[i,'Chapter'] = random.choice(['Other US','Texas','Bay Area','DC Area','New England'])

In [185]:
df_SAA[df_SAA.first_name == 'Mary']

,pref_mail_name,pref_class_year,home_city,home_state_code,home_country,home_phone_area_code,home_phone_number,home_email_address,bus_city,bus_state_code,bus_country,bus_phone_area_code,bus_phone_number,bus_email_address,first_name,last_name,pref_name_sort,email_switch,saa_email_address,gsb_email_address,other_email_address,pref_phone_area_code,pref_phone_number,pref_phone_addr_type,memb_status_desc,short_degree_string,parent_degree_string,short_degree_string_spouse,parent_degree_string_spouse,primary_sort_name,plan_name,primary_ind,0
1,NaN,NaN,Madrid,NaN,Japan,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,Mary,Rodriguez,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,"MA '85, BS '92",NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,Madrid,NaN,NaN,NaN,*,mdavis343@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,Mary,Davis,NaN,NaN,m.davis4650@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,London,NaN,UAE,NaN,*,marymiller5944@gmail.com,NaN,NaN,Kuwait,NaN,NaN,mary.miller5372@alumni.stanford.edu,Mary,Miller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"BA '94, JD '83, MA '80",NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,Beijing,NaN,UAE,NaN,NaN,mary.rodriguez1968@stanfordalumni.org,NaN,NaN,China,NaN,NaN,NaN,Mary,Rodriguez,NaN,NaN,mary.rodriguez9518@stanfordalumni.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [186]:
# 5th record. Changed last name
rec = df_SAA.loc[12]
i=4
df_mc.loc[i,'First Name'] = rec.first_name
df_mc.loc[i,'Last Name'] = random.choice(common_LNames)
df_mc.loc[i,'Email Address'] = rec.saa_email_address.split('@')[0]+'@gmail.com'
df_mc.loc[i,'Country'] = 'USA'
df_mc.loc[i,'Degree'] = random.choice([np.nan,''])

df_mc.loc[i,'Board Member'] = random.choice([True,False])
df_mc.loc[i,'Gender'] = random.choice(['F','M',np.nan])
df_mc.loc[i,'Chapter'] = random.choice(['Other US','Texas','Bay Area','DC Area','New England'])

In [194]:
df_mc

,Email Address,First Name,Last Name,Board Member,Gender,Chapter,Reunion Year,Country,Degree,MEMBER_RATING,OPTIN_TIME,OPTIN_IP,CONFIRM_TIME,CONFIRM_IP,LATITUDE,LONGITUDE,GMTOFF,DSTOFF,TIMEZONE,CC,REGION,CLEAN_TIME,CLEAN_CAMPAIGN_TITLE,CLEAN_CAMPAIGN_ID,LEID,EUID,NOTES,TAGS,0
0,brodriguez369@gmail.com,Barbara,Rodriguez,True,NaN,New England,NaN,USA,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,liamjones7027@stanfordalumni.org,Liam,Jones,False,M,Bay Area,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,lindam7966@alumni.stanford.edu,Liam,Rodriguez,True,NaN,Bay Area,NaN,USA,MD;MA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,liam.garcia7988@stanfordalumni.org,Liam,Garcia,True,F,New England,NaN,USA,MS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,m.davis4650@gmail.com,Mary,Johnson,True,NaN,Texas,NaN,USA,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
df_mc.drop(columns = [0]).to_csv('data/Fake_MailChimp_cleaned.csv',index=False)

In [202]:
['alt']  * df_mc.shape[0]

['alt', 'alt', 'alt', 'alt', 'alt']

In [198]:
df_mc

,Email Address,First Name,Last Name,Board Member,Gender,Chapter,Reunion Year,Country,Degree,MEMBER_RATING,OPTIN_TIME,OPTIN_IP,CONFIRM_TIME,CONFIRM_IP,LATITUDE,LONGITUDE,GMTOFF,DSTOFF,TIMEZONE,CC,REGION,CLEAN_TIME,CLEAN_CAMPAIGN_TITLE,CLEAN_CAMPAIGN_ID,LEID,EUID,NOTES,TAGS,0
0,brodriguez369@gmail.com,Barbara,Rodriguez,True,NaN,New England,NaN,USA,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,liamjones7027@stanfordalumni.org,Liam,Jones,False,M,Bay Area,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,lindam7966@alumni.stanford.edu,Liam,Rodriguez,True,NaN,Bay Area,NaN,USA,MD;MA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,liam.garcia7988@stanfordalumni.org,Liam,Garcia,True,F,New England,NaN,USA,MS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,m.davis4650@gmail.com,Mary,Johnson,True,NaN,Texas,NaN,USA,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [201]:
df_mc.shape

(5, 29)

In [204]:
df_mc['new'] = ['alt']  * df_mc.shape[0]

In [205]:
df_mc

,Email Address,First Name,Last Name,Board Member,Gender,Chapter,Reunion Year,Country,Degree,MEMBER_RATING,OPTIN_TIME,OPTIN_IP,CONFIRM_TIME,CONFIRM_IP,LATITUDE,LONGITUDE,GMTOFF,DSTOFF,TIMEZONE,CC,REGION,CLEAN_TIME,CLEAN_CAMPAIGN_TITLE,CLEAN_CAMPAIGN_ID,LEID,EUID,NOTES,TAGS,0,new
0,brodriguez369@gmail.com,Barbara,Rodriguez,True,NaN,New England,NaN,USA,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,alt
1,liamjones7027@stanfordalumni.org,Liam,Jones,False,M,Bay Area,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,alt
2,lindam7966@alumni.stanford.edu,Liam,Rodriguez,True,NaN,Bay Area,NaN,USA,MD;MA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,alt
3,liam.garcia7988@stanfordalumni.org,Liam,Garcia,True,F,New England,NaN,USA,MS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,alt
4,m.davis4650@gmail.com,Mary,Johnson,True,NaN,Texas,NaN,USA,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,alt


In [206]:
df_SAA 

,pref_mail_name,pref_class_year,home_city,home_state_code,home_country,home_phone_area_code,home_phone_number,home_email_address,bus_city,bus_state_code,bus_country,bus_phone_area_code,bus_phone_number,bus_email_address,first_name,last_name,pref_name_sort,email_switch,saa_email_address,gsb_email_address,other_email_address,pref_phone_area_code,pref_phone_number,pref_phone_addr_type,memb_status_desc,short_degree_string,parent_degree_string,short_degree_string_spouse,parent_degree_string_spouse,primary_sort_name,plan_name,primary_ind,0
0,NaN,NaN,London,NaN,NaN,NaN,NaN,*,NaN,NaN,Kuwait,NaN,NaN,NaN,Linda,Brown,NaN,NaN,lindab1567@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,"PhD '94, BA '82",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Madrid,NaN,Japan,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,Mary,Rodriguez,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,"MA '85, BS '92",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Shanghai,NaN,USA,NaN,898 7073-217,NaN,NaN,NaN,UAE,NaN,NaN,NaN,Sarah,Rodriguez,NaN,NaN,sarah.rodriguez992@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,"BS '88, BS '95, MA '85",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Chicago,IL,UAE,NaN,NaN,NaN,NaN,NaN,China,NaN,NaN,NaN,Jessica,Jones,NaN,NaN,jessicaj5803@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,MBA '81,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1994.0,Seoul,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jessica,Martinez,NaN,NaN,jessicamartinez1707@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,2003.0,Shanghai,NaN,USA,NaN,*,NaN,NaN,NaN,UAE,NaN,NaN,NaN,Elizabeth,Miller,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"JD '98, JD '98, BS '85",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,Shanghai,NaN,*,NaN,*,lindam7966@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,Linda,Martinez,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,"MS '99, JD '84",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,Seoul,NaN,Japan,NaN,NaN,NaN,NaN,NaN,*,NaN,NaN,NaN,Lucas,Miller,NaN,NaN,*,NaN,NaN,NaN,NaN,NaN,NaN,"BS '83, MS '94, PhD '82",NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,Tokyo,NaN,*,NaN,*,*,NaN,NaN,United States,NaN,NaN,NaN,Jennifer,Garcia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"'80, MD '98",NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,1991.0,Seoul,NaN,Kuwait,NaN,*,NaN,NaN,NaN,Japan,NaN,NaN,NaN,Henry,Brown,NaN,NaN,henryb2588@alumni.stanford.edu,NaN,NaN,NaN,NaN,NaN,NaN,"'96, MA '99",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
import pycountry

In [211]:
list(pycountry.countries)[0]

Country(alpha_2='AW', alpha_3='ABW', flag='🇦🇼', name='Aruba', numeric='533')

In [220]:
pycountry.countries.lookup('ind')

Country(alpha_2='IN', alpha_3='IND', flag='🇮🇳', name='India', numeric='356', official_name='Republic of India')